# Clase de scraping de Fotocasa.es
El siguiente código permite scrapear las propiedades de Fotocasa.es de diferentes maneras. Utilizando las librerías Selenium y BeautifulSoup para obtener la data de la propia web y concurrent.futures para acelerar la obtención, podemos conseguir diferentes datasets para la ciudad y el rango de páginas que especifiquemos como parámetros.

De igual modo, el script contempla tres modos de uso, uno que guarda el dato en un dataframe directamente, otro que lo guarda y además descarga cada propiedad en un archivo de texto, y un tercero que nos permite descargar o no y crear el dataframe desde la descarga realizada en el momento o previamente; este último modo nos permite trabajar con archivos en nuestro disco duro sin necesidad de volver a recorrer todas las páginas de la web.

La clase finalmente guarda el dataframe resultante en un archivo .csv por si queremos utilizar el dato más adelante.

In [1]:
%%time

from selenium.webdriver.common.keys import Keys
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.action_chains import ActionChains
#from selenium.webdriver.common.by import By
#from selenium.webdriver.support.ui import WebDriverWait
#from selenium.webdriver.support import expected_conditions as EC
from random_user_agent.user_agent import UserAgent
from random_user_agent.params import SoftwareName, OperatingSystem
import undetected_chromedriver as uc

from bs4 import BeautifulSoup
import concurrent.futures
import os
import shutil
import numpy as np
import requests
import re
import time
import pandas as pd
import json

software_name = [SoftwareName.CHROME.value]
operating_systems = [OperatingSystem.WINDOWS.value,OperatingSystem.LINUX.value]
user_agent_rotator = UserAgent(software_names=software_name,
                               operating_systems=operating_systems,
                               limit=100)
user_agent = user_agent_rotator.get_random_user_agent()

chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
chrome_options.add_argument('--enable-javascript')
chrome_options.add_argument(f'user-agent={user_agent}')


class fotocasa_scraping:
  '''
  Clase para inicializar el scraping de Fotocasa.es
  '''

  def check_features(self, data, page):
    '''
    Función de comprobación y transformación de las features extraídas de cada
    vivienda. En el proceso se comprueba su existencia y se transforman en función
    de los ids asignados por Fotocasa. Si no existe el campo se rellena con un
    circunflejo (^).
    
    Args:
    - data (json): json contenedor de las features de cada vivienda.
    - page (int): número de la página.
    ''' 
  
    # Diccionario de features
    realestate = {
        'origin': 'fotocasa',
        'page': page,
        'title': '',
        'link': '',
        'image_url': '',
        'country': '',
        'district': '',
        'neighborhood': '',
        'street': '',
        'zipCode': '',
        'province': '',
        'buildingType': '',
        'clientAlias': '',
        'latitude': '',
        'longitude': '',
        'isNewConstruction': '',
        'rooms': '',
        'bathrooms': '',
        'parking': '',
        'elevator': '',
        'furnished': '',
        'surface': '',
        'energyCertificate': '',
        'hotWater': '',
        'heating': '',
        'conservationState': '',
        'antiquity': '',
        'floor': '',
        'surfaceLand': '',
        'otherFeatures': '',
        'price': '',     
        }
    # Comienzan las comprobaciones feature a feature
    try:
        realestate['title'] = data['propertyTitle']    
    except:
        realestate['title'] = '^'
        
    try:
        realestate['link'] = 'https://www.fotocasa.es' + data['realEstate']['detail']['es-ES']   
    except:
        realestate['link'] = '^'

    try:
        realestate['image_url'] = data['realEstate']['multimedia'][1]['src']
    except:
        realestate['image_url'] = '^'
        
    try:
        realestate['country'] = data['realEstate']['address']['country']
    except:
        realestate['country'] = '^'
        
    try:
        realestate['district'] = data['realEstate']['address']['district']
    except:
        realestate['district'] = '^'
        
    try:
        realestate['neighborhood'] = data['realEstate']['address']['neighborhood']
    except:
        realestate['neighborhood'] = '^'
        
    try:
        realestate['street'] = data['realEstate']['location']
    except:
        realestate['street'] = '^'
        
    try:
        realestate['zipCode'] = data['realEstate']['address']['zipCode']
    except:
        realestate['zipCode'] = '^'
        
    try:
        realestate['province'] = data['realEstate']['address']['province']
    except:
        realestate['province'] = '^'
        
    try:
        realestate['buildingType'] = data['realEstate']['buildingType']
    except:
        realestate['buildingType'] = '^'

    try:
        realestate['clientAlias'] = data['realEstate']['clientAlias']
    except:
        realestate['clientAlias'] = '^'
        
    try:
        realestate['latitude'] = data['realEstate']['coordinates']['latitude']
    except:
        realestate['latitude'] = '^'

    try:
        realestate['longitude'] = data['realEstate']['coordinates']['longitude']
    except:
        realestate['longitude'] = '^'
        
    try:
        realestate['isNewConstruction'] = data['realEstate']['isNewConstruction']
    except:
        realestate['isNewConstruction'] = '^'
        
    try:
        realestate['rooms'] = data['realEstate']['features']['rooms']
    except:
        realestate['rooms'] = '^'
        
    try:
        realestate['bathrooms'] = data['realEstate']['features']['bathrooms']
    except:
        realestate['bathrooms'] = '^'

    try:
        featureList = data['realEstate']['featuresList']
        realestate['parking'] = ''.join([featureList[index]['value'] for index,value in enumerate(featureList) if featureList[index]['label'] == 'parking'])
        
    except:
        realestate['parking'] = '^'

    try:
        featureList = data['realEstate']['featuresList']
        realestate['elevator'] = ''.join([featureList[index]['value'] for index,value in enumerate(featureList) if featureList[index]['label'] == 'elevator'])
        
    except:
        realestate['elevator'] = '^'

    try:
        featureList = data['realEstate']['featuresList']
        realestate['furnished'] = ''.join([featureList[index]['value'] for index,value in enumerate(featureList) if featureList[index]['label'] == 'furnished'])
        
    except:
        realestate['furnished'] = '^'
        
    try:
        realestate['surface'] = data['realEstate']['features']['surface']
    except:
        realestate['surface'] = '^'
        
    try:
        realestate['energyCertificate'] = data['realEstate']['energyCertificate']
    except:
        realestate['energyCertificate'] = '^'
        
    try:
        realestate['hotWater'] = data['realEstate']['features']['hotWater']
        featureList = data['realEstate']['featuresList']
        realestate['hotWater'] = ''.join([featureList[index]['value'] for index,value in enumerate(featureList) if featureList[index]['label'] == 'hotWater'])
        
    except:
        realestate['hotWater'] = '^'
        
    try:
        realestate['heating'] = data['realEstate']['features']['heating']
        featureList = data['realEstate']['featuresList']
        realestate['heating'] = ''.join([featureList[index]['value'] for index,value in enumerate(featureList) if featureList[index]['label'] == 'heating'])
       
    except:
        realestate['heating'] = '^'
        
    try:
        realestate['conservationState'] = data['realEstate']['features']['conservationState']
        featureList = data['realEstate']['featuresList']
        realestate['conservationState'] = ''.join([featureList[index]['value'] for index,value in enumerate(featureList) if featureList[index]['label'] == 'conservationState'])
       
    except:
        realestate['conservationState'] = '^'
        
    try:
        realestate['antiquity'] = data['realEstate']['features']['antiquity']
        featureList = data['realEstate']['featuresList']
        realestate['antiquity'] = ''.join([featureList[index]['value'] for index,value in enumerate(featureList) if featureList[index]['label'] == 'antiquity'])
       
    except:
        realestate['antiquity'] = '^'
        
    try:
        realestate['floor'] = data['realEstate']['features']['floor']
    except:
        realestate['floor'] = '^'
        
    try:
        realestate['surfaceLand'] = data['realEstate']['features']['surfaceLand']
    except:
        realestate['surfaceLand'] = '^'
        
    try:
        realestate['otherFeatures'] = data['realEstate']['otherFeatures']
    except:
        realestate['otherFeatures'] = '^'
        
    try:
        realestate['price'] = data['realEstate']['price']
    except:
        realestate['price'] = 0
        
    #devuelve un diccionario
    return realestate

#--------------------------------------

  def parse_properties(self, url_list, page, download, files_mode):
    '''
    Extracción del fragmento de código javascript de cada página de vivienda
    para convertirlo en json y extraer posteriormente las features. En este proceso
    se limpia el código con expresiones regulares y se gestionan los modos activados.
    
    Args:
    - url_list (list): lista de urls de viviendas que se han extraído de la página actual.
    - page (int): número de la página.
    - download (bool): booleano para activar la descarga de las propiedades en el disco duro (True).
    - files_mode (bool): booleano para activar el modo de lectura de archivos previamente descargados (True).
    ''' 
    df_page = pd.DataFrame()
        
    i = 0
    
    if download == True:
        directory = f'fotocasa/fotocasa_{page}' # Creo una carpeta por cada página
        os.mkdir(directory)
    
    for url in url_list:
        time.sleep(1)
        #print(url)
        headers = {'User-Agent': user_agent}
        r = requests.get(url, headers=headers)
        soup = BeautifulSoup(r.content,'html.parser')
        
        # Obtención y limpieza de la fila del script que contiene la información de propiedades
        prop_scripts = soup.findAll('script')
        prop_features = ''.join([re.search('window.__INITIAL_PROPS__ = JSON.parse(.*)\n',str(x)).group(1) for x in prop_scripts if re.search('window.__INITIAL_PROPS__',str(x))])
        prop_features_clean = re.sub(r'\\"','"',prop_features)
        prop_features_clean = re.sub(r'\\\\"','',prop_features_clean)
        prop_features_clean = re.sub(r'\("|"\);','',prop_features_clean)
        prop_features_clean = re.sub(r',"seo":.*','}',prop_features_clean)
        
        try:
            prop_data = json.loads(prop_features_clean)
            realestate = self.check_features(prop_data,page)
            
            if download == True:
                self.download_realestates(prop_features_clean,page,i)
                i = i + 1
                
            if files_mode == False:
                df = pd.DataFrame([realestate])
                df_page = pd.concat([df_page,df],ignore_index=True)
              
        except:
            print(f'Error en página {page}: {url}')
        
    return df_page

#--------------------------------------

  def download_realestates(self,realestate,page,num):
    '''
    Guardado de cada vivienda en archivos diferentes dentro de la carpeta con la página.
    El nombre del archivo contiene la página y el número de la vivienda dentro de ésta.
    
    Args:
    - realstate (str): fragmento del código de la página del cual se extraen los datos para las features.
    - page (int): número de la página.
    - num (int): número de la vivienda dentro de la página.
    ''' 
        
    f = open('fotocasa/fotocasa_%s/realestate_%s_%s' % (page,page,num), 'w') # la W es para permisos de writing (escritura)
    f.write(realestate)
    f.close()
    
    return None

#--------------------------------------

  def property_list(self, province, page):
    '''
    Extracción de la lista de urls de cada una de las páginas de parrillas de viviendas.
    
    Args:
    - province (str): nombre de la provincia a scrapear.
    - page (int): número de la página.
    ''' 
    # print(f'Entra en página {page}')
    # Recibo un número de página. Almaceno las urls de todas las propiedades de cada página de parrilla. Devuelvo una lista de urls.
    
    headers = {'User-Agent': user_agent}
    #r = requests.get(f'https://www.fotocasa.es/es/comprar/viviendas/{province}-provincia/todas-las-zonas/l/{str(page)}', headers=headers)
    r = requests.get('https://www.fotocasa.es/es/comprar/viviendas/madrid-provincia/madrid-zona-de/l' + '/' + str(page), headers=headers)
    soup = BeautifulSoup(r.content,'html.parser')
    
    # Obtención y limpieza de la fila del script que contiene la información de propiedades
    prop_scripts = soup.findAll('script')
    prop_pages = ''.join([re.search('window.__INITIAL_PROPS__ = JSON.parse(.*)\n',str(x)).group(1) for x in prop_scripts if re.search('window.__INITIAL_PROPS__',str(x))])
    prop_pages_clean = re.sub(r'\\"','"',prop_pages)
    prop_pages_clean = re.sub(r'\\\\"','',prop_pages_clean)
    prop_pages_clean = re.sub(r'\("|"\);','',prop_pages_clean)
    prop_pages_clean = re.sub(r',"seo":.*','}',prop_pages_clean)
    prop_data = json.loads(prop_pages_clean)
    
    property_url_list = []
    try:
        [property_url_list.append(f'https://www.fotocasa.es{realestate["detail"]["es-ES"]}') for realestate in list(prop_data['initialSearch']['result']['realEstates'])]
    except:
        pass
    
    print(f'Almacena {len(property_url_list)} urls de la página {page}')

    return property_url_list

#--------------------------------------

  def pages_to_scrape(self, driver, province):
    '''
    Utilizando Selenium se hace una llamada a la página de parrilla de viviendas,
    se va haciendo scroll down y buscando la clase de css del paginado de la página
    hasta encontrarlo y aislarlo.
    
    Args:
    - driver (uc): driver de Selenium creado con undetected Chrome.
    - province (str): nombre de la provincia a scrapear.
    ''' 
    # Obtengo el número de páginas totales que debo recorrer utilizando Selenium, ya que es necesario llegar al footer.

    driver.get(f'https://www.fotocasa.es/es/comprar/viviendas/madrid-provincia/madrid-zona-de/l')
    #driver.get(f'https://www.fotocasa.es/es/comprar/viviendas/{province}-provincia/todas-las-zonas/l')
    
    page_selector = []
    while len(page_selector) < 1:
        html_txt = driver.page_source
        soup = BeautifulSoup(html_txt,'html.parser')
        page_selector = soup.findAll('li',attrs={'class':'sui-MoleculePagination-item'})
        ActionChains(driver).key_down(Keys.PAGE_DOWN).perform()
        time.sleep(0.1)
       
    n_pages = re.search('<span class="sui-AtomButton-inner">(.*)</span>',str(page_selector[-2])).group(1)
        
    return int(n_pages)

#--------------------------------------

  def divide_pages(self, page_range):
    '''
    División de los rangos de páginas a scrapear en 5 grupos. Calcula los percentiles
    [0, 20, 40, 60, 80] del rango de páginas y los utiliza para crear los 5 grupos
    con el mismo número de páginas
    
    Args:
    - page_range (list): lista de dos valores enteros, el inicial y el final.
    ''' 
    if page_range[1]-page_range[0] > 10:
        pages = []
        num_pages = list(range(page_range[0], page_range[1]))
        percentiles = [int(np.percentile(num_pages,x)) for x in range(0, 100, 20)]

        for loc in range(len(percentiles)):
            if percentiles[loc] == page_range[0]:
                start = page_range[0]
            else:
                start = percentiles[loc]+1

            if loc == len(percentiles)-1:
                end = page_range[1]
            else:
                end = percentiles[loc+1]

            pages.append([start, end+1])
            
        return pages
    else:
        return [[page_range[0],page_range[1]+1]]
    
    return None

#--------------------------------------

  def scraping_loop(self, page_range, province, download, files_mode):
    '''
    Bucle principal de scraping donde se extraen las urls existentes en cada parrilla
    de viviendas y luego se pasan a la función que extrae los datos de las mismas.
    
    Args:
    - page_range (list): lista de dos valores enteros, el inicial y el final.
    - province (str): nombre de la provincia que se desea scrapear.
    - download (bool): booleano para activar la descarga de las propiedades en el disco duro (True).
    - files_mode (bool): booleano para activar el modo de lectura de archivos previamente descargados (True).
    '''    
    data_d = pd.DataFrame()
    
    for page in range(page_range[0], page_range[1]):
        time.sleep(0.5)
        properties_per_page = self.property_list(province, page)
        property_data = self.parse_properties(properties_per_page, page, download, files_mode)

        data_d = pd.concat([data_d,property_data], ignore_index=True)

        print(f'Página {str(page)} terminada.')
        
    return data_d

#--------------------------------------

  def files_mode(self):
    '''
    Gestión del modo lectura. Comprueba que existe el directorio 'fotocasa' y las subcarpetas
    con los números de página. A continuación lista los archivos dentro de cada una y comienza
    a leerlos en formato json, comprueba las features y lo concatena en un dataframe.
    
    Args:
    - No necesita argumentos de entrada.
    '''
    
    path = os.getcwd()
    directory = 'fotocasa'
    df_files = pd.DataFrame()

    if os.path.exists(directory):
        os.chdir(directory)

        for page_number in os.listdir():
            if re.search('^fotocasa.*',page_number):
                page_directory = f'{page_number}'
            else:
                continue
            
            if os.path.exists(page_directory):
                os.chdir(page_directory) 
                
                for file in os.listdir():
                    if re.search('^realestate.*',file):
                        page = re.sub('_','',re.search('_(.*)_',file).group())
                        with open(f'{path}/{directory}/{page_directory}/{file}', 'r') as f:
                            prop_data = json.loads(f.read())
                            realestate = self.check_features(prop_data,page)

                            df = pd.DataFrame([realestate])
                            df_files = pd.concat([df_files,df],ignore_index=True)
            os.chdir(f'{path}/{directory}')
    else:
        print(f'No existe el directorio {directory}.')
    
    os.chdir(path)
    #print(os.getcwd())
                    
    return df_files

#--------------------------------------

  def concurrent_scraping(self, page_range, province, download, files_mode):
    '''
    Función que pone en marcha el bucle de concurrencia para acelerar el scraping.
    Utilizando concurrent_futures crea 5 grupos a los que se van asignando los rangos de páginas.
    Después del scraping concatena todos los dataframes extraidos por los procesos paralelos.
    
    Args:
    - page_range (list): lista de listas que contienen los diferentes rangos de páginas ya divididos. Será 1 si hay menos de 10 páginas y 5 si hay más.
    - province (str): nombre de la provincia que se desea scrapear.
    - download (bool): booleano para activar la descarga de las propiedades en el disco duro (True).
    - files_mode (bool): booleano para activar el modo de lectura de archivos previamente descargados (True).
    '''
    data = pd.DataFrame()
    
    if len(page_range) > 1:
        with concurrent.futures.ThreadPoolExecutor(max_workers=5) as executor:
            futures = []

            for r in page_range:
                futures.append(executor.submit(self.scraping_loop, page_range = r, province = province, download = download, files_mode = files_mode))

            for future in concurrent.futures.as_completed(futures):
                data = pd.concat([data,future.result()], ignore_index = True)
    else:
        data = self.scraping_loop(page_range = page_range[0], province = province, download = download, files_mode = files_mode)
            
    return data

#--------------------------------------

  def check_range(self, page_range, total_pages):
    '''
    Recibe el rango de páginas introducido por el usuario para hacer comprobaciones.
    Si el valor es el default, scrapea todas las páginas de la provincia.
    Si el valor final es mayor que el total de páginas, lo iguala al total de páginas.
    Si el valor inicial es mayor que el final, le da la vuelta.
    
    Args:
    - page_range (list): lista de dos valores enteros que son la página inicial y la final a scrapear.
    - total_pages (int): número total de páginas que tiene la provincia.
    '''
    if page_range[0] == 0 and page_range[1] == 0:
        page_range[0] = 1
        page_range[1] = total_pages

    if page_range[1] > total_pages:
        page_range[1] = total_pages

    if page_range[0] > page_range[1]:
        aux = page_range[0]
        page_range[0] = page_range[1]
        page_range[1] = aux

    return page_range
#--------------------------------------


  def __init__(self, province = 'Madrid', page_range = [0,0], download = False, files_mode = False):
    '''
    Función inicial de la clase que recibe los parámetros que ponen en marcha el scraping.
    Recibe el nombre de la provincia, el rango de páginas y el estado de los modos de
    descarga y lectura de archivos.
    
    Args:
    - province (str): nombre de la provincia que se desea scrapear
    - page_range ([int,int]): lista de dos números enteros, la página inicial y la página final.
    - download (bool): booleano para activar la descarga de las propiedades en el disco duro (True).
    - files_mode (bool): booleano para activar el modo de lectura de archivos previamente descargados (True).
    '''
    self.data = pd.DataFrame()
    directory = 'fotocasa'

    # GESTIÓN INICIAL DE RANGOS DE PÁGINAS A SCRAPEAR
    if files_mode == False or (files_mode == True and download == True):
        
        driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()),options=chrome_options)
        total_pages = self.pages_to_scrape(driver, province) # Extraigo el máximo de páginas a scrapear de la parrilla de la ciudad
        page_range = self.check_range(page_range,total_pages) # Compruebo y corrijo posibles errores en los rangos de páginas introducidos o el valor por defecto
        print('Total de páginas: ' + str(page_range[1]-page_range[0]))
        
        page_range = self.divide_pages(page_range) # Divido el rango introducido en 5 grupos iguales si hay más de 10 páginas
        print(f'Los rangos a scrapear son: {page_range}')

    # GESTIÓN DE LOS MODOS
    if files_mode == True:
        print('> Modo lectura activado.')
        if download == True:
            print('> Modo descarga activado.')
            if os.path.exists(directory):
                shutil.rmtree(directory) # si existe borro la carpeta y todo su contenido
            os.mkdir(directory)
            self.data = self.concurrent_scraping(page_range, province, download, files_mode)
            self.data = self.files_mode()
        else:
            try:
                self.data = self.files_mode()
            except:
                print('Ha habido un error en la lectura de los archivos. Comprueba que existe el directorio.')
            
        print('> Done!')
        self.data.to_csv('concurrent_scraping_from_files.csv',index=False)
        
    else:
        if download == True:
            print('> Modo descarga activado.')
            if os.path.exists(directory):
                shutil.rmtree(directory) # si existe borro la carpeta y todo su contenido
            os.mkdir(directory)

        self.data = self.concurrent_scraping(page_range, province, download, files_mode)

        print('> Done!')
        self.data.to_csv(f'concurrent_scraping_from_dataframe.csv',index=False)
        
    return None

CPU times: user 881 ms, sys: 109 ms, total: 990 ms
Wall time: 1.35 s


In [2]:
%%time
fc_scraping_madrid = fotocasa_scraping(province = 'Madrid', download = True, files_mode = False)


Total de páginas: 521
Los rangos a scrapear son: [[1, 106], [106, 210], [210, 314], [314, 418], [418, 523]]
> Modo descarga activado.
Almacena 30 urls de la página 418
Almacena 30 urls de la página 210
Almacena 30 urls de la página 106
Almacena 30 urls de la página 1
Almacena 30 urls de la página 314
Página 1 terminada.
Página 418 terminada.
Página 210 terminada.
Página 106 terminada.
Almacena 30 urls de la página 419
Página 314 terminada.
Almacena 30 urls de la página 2
Almacena 30 urls de la página 211
Almacena 30 urls de la página 107
Almacena 30 urls de la página 315
Página 2 terminada.
Página 107 terminada.
Almacena 30 urls de la página 3
Página 211 terminada.
Página 419 terminada.
Almacena 30 urls de la página 108
Página 315 terminada.
Almacena 30 urls de la página 420
Almacena 30 urls de la página 212
Almacena 30 urls de la página 316
Página 420 terminada.
Página 212 terminada.
Almacena 30 urls de la página 421
Página 108 terminada.
Página 3 terminada.
Almacena 30 urls de la pá

Almacena 30 urls de la página 343
Página 29 terminada.
Almacena 30 urls de la página 135
Almacena 30 urls de la página 30
Página 447 terminada.
Almacena 30 urls de la página 448
Página 135 terminada.
Página 239 terminada.
Almacena 30 urls de la página 136
Página 343 terminada.
Almacena 30 urls de la página 344
Almacena 30 urls de la página 240
Página 30 terminada.
Almacena 30 urls de la página 31
Página 448 terminada.
Almacena 30 urls de la página 449
Página 136 terminada.
Página 344 terminada.
Almacena 30 urls de la página 345
Almacena 30 urls de la página 137
Página 240 terminada.
Página 31 terminada.
Almacena 30 urls de la página 241
Almacena 30 urls de la página 32
Página 449 terminada.
Almacena 30 urls de la página 450
Página 137 terminada.
Almacena 30 urls de la página 138
Página 241 terminada.
Página 345 terminada.
Almacena 30 urls de la página 242
Almacena 30 urls de la página 346
Página 32 terminada.
Almacena 30 urls de la página 33
Página 450 terminada.
Almacena 30 urls de la

Almacena 30 urls de la página 268
Página 164 terminada.
Almacena 30 urls de la página 165
Página 477 terminada.
Almacena 30 urls de la página 478
Página 372 terminada.
Almacena 30 urls de la página 373
Página 59 terminada.
Almacena 30 urls de la página 60
Página 268 terminada.
Almacena 30 urls de la página 269
Página 165 terminada.
Almacena 30 urls de la página 166
Página 478 terminada.
Almacena 30 urls de la página 479
Página 373 terminada.
Almacena 30 urls de la página 374
Página 60 terminada.
Almacena 30 urls de la página 61
Página 269 terminada.
Almacena 30 urls de la página 270
Página 166 terminada.
Almacena 30 urls de la página 167
Página 479 terminada.
Almacena 30 urls de la página 480
Página 374 terminada.
Almacena 30 urls de la página 375
Página 61 terminada.
Almacena 30 urls de la página 62
Página 270 terminada.
Almacena 30 urls de la página 271
Página 167 terminada.
Almacena 30 urls de la página 168
Página 480 terminada.
Almacena 30 urls de la página 481
Página 375 terminada

Página 193 terminada.
Almacena 30 urls de la página 194
Página 88 terminada.
Página 297 terminada.
Almacena 30 urls de la página 89
Almacena 30 urls de la página 298
Página 402 terminada.
Almacena 30 urls de la página 403
Página 507 terminada.
Almacena 30 urls de la página 508
Página 194 terminada.
Almacena 30 urls de la página 195
Página 89 terminada.
Página 298 terminada.
Almacena 30 urls de la página 90
Almacena 30 urls de la página 299
Página 403 terminada.
Almacena 30 urls de la página 404
Página 508 terminada.
Almacena 30 urls de la página 509
Página 195 terminada.
Almacena 30 urls de la página 196
Página 90 terminada.
Almacena 30 urls de la página 91
Página 299 terminada.
Página 404 terminada.
Almacena 30 urls de la página 300
Almacena 30 urls de la página 405
Página 509 terminada.
Almacena 30 urls de la página 510
Página 196 terminada.
Almacena 30 urls de la página 197
Página 91 terminada.
Almacena 30 urls de la página 92
Página 300 terminada.
Página 405 terminada.
Almacena 30 

In [3]:
fc_scraping_madrid.data.shape

(16200, 31)

In [5]:
fc_scraping_madrid.data.sample(5)

,origin,page,title,link,image_url,country,district,neighborhood,street,zipCode,...,surface,energyCertificate,hotWater,heating,conservationState,antiquity,floor,surfaceLand,otherFeatures,price
10669,fotocasa,140,Piso en venta en Calle de Arenas,https://www.fotocasa.es/es/comprar/vivienda/ma...,https://static.inmofactory.com/images/inmofact...,España,Villaverde,San Andrés,Calle de Arenas,28021,...,70,G,Electricidad,,Bien,50 a 70 años,8,0,"{'6': 'Gres Cerámica', '10': 'Terraza', '21': ...",110000
13418,fotocasa,16,Piso en venta en Plaza de Vulcano,https://www.fotocasa.es/es/comprar/vivienda/ma...,https://static.inmofactory.com/images/inmofact...,España,Carabanchel,Abrantes,Plaza de Vulcano,28025,...,77,G,Gas Natural,Gas Natural,Muy bien,50 a 70 años,15,0,"{'1': 'Aire acondicionado', '2': 'Armarios', '...",160000
9088,fotocasa,519,Piso en venta en Calle de Medellín,https://www.fotocasa.es/es/comprar/vivienda/ma...,https://static.inmofactory.com/images/inmofact...,España,Chamberí,Trafalgar,Calle de Medellín,28010,...,57,G,,,Casi nuevo,70 a 100 años,9,0,"{'1': 'Aire acondicionado', '2': 'Armarios', '...",369000
11732,fotocasa,176,Piso en venta en Calle del General Lacy,https://www.fotocasa.es/es/comprar/vivienda/ma...,https://static.inmofactory.com/images/inmofact...,España,Arganzuela,Palos de Moguer,Calle del General Lacy,28045,...,58,E,,,,+ 100 años,8,0,{'32': 'Balcón'},212000
7030,fotocasa,451,Piso en venta en Paseo de la Castellana,https://www.fotocasa.es/es/comprar/vivienda/ma...,https://static.inmofactory.com/images/inmofact...,España,Tetuán,Almenara -Ventilla,Paseo de la Castellana,28046,...,110,G,,,Bien,,7,0,"{'2': 'Armarios', '3': 'Calefacción', '9': 'Pa...",730000


## Modo por defecto con rango + descarga de propiedades

In [11]:
%%time
fc_scraping = fotocasa_scraping(province = 'Madrid', page_range = [20,30], download = True, files_mode = False)


Total de páginas: 10
Los rangos a scrapear son: [[20, 31]]
> Modo descarga activado.
Almacena 30 urls de la página 20
Página 20 terminada.
Almacena 30 urls de la página 21
Página 21 terminada.
Almacena 30 urls de la página 22
Página 22 terminada.
Almacena 30 urls de la página 23
Página 23 terminada.
Almacena 30 urls de la página 24
Página 24 terminada.
Almacena 30 urls de la página 25
Página 25 terminada.
Almacena 30 urls de la página 26
Página 26 terminada.
Almacena 30 urls de la página 27
Página 27 terminada.
Almacena 30 urls de la página 28
Página 28 terminada.
Almacena 30 urls de la página 29
Página 29 terminada.
Almacena 30 urls de la página 30
Página 30 terminada.
> Done!
CPU times: user 39 s, sys: 4.12 s, total: 43.1 s
Wall time: 9min 49s


In [13]:
fc_scraping.data.shape

(330, 31)

In [14]:
fc_scraping.data.head(10)

,origin,page,title,link,image_url,country,district,neighborhood,street,zipCode,...,surface,energyCertificate,hotWater,heating,conservationState,antiquity,floor,surfaceLand,otherFeatures,price
0,fotocasa,20,Piso en venta en Calle de Los Tudescos,https://www.fotocasa.es/es/comprar/vivienda/ma...,https://static.inmofactory.com/images/inmofact...,España,Centro,Universidad - Malasaña,Calle de Los Tudescos,28004,...,136,G,,,,70 a 100 años,8,0,"{'1': 'Aire acondicionado', '2': 'Armarios', '...",730000
1,fotocasa,20,Piso en venta,https://www.fotocasa.es/es/comprar/vivienda/ma...,https://static.inmofactory.com/images/inmofact...,España,Chamberí,Ríos Rosas - Nuevos Ministerios,,28003,...,108,G,,Gas Natural,Bien,70 a 100 años,7,0,"{'1': 'Aire acondicionado', '2': 'Armarios', '...",900000
2,fotocasa,20,Piso en venta en Carril de Los Caleros,https://www.fotocasa.es/es/comprar/vivienda/ma...,https://static.inmofactory.com/images/inmofact...,España,Moncloa - Aravaca,Aravaca,Carril de Los Caleros,28023,...,275,G,,,,,6,0,"{'1': 'Aire acondicionado', '2': 'Armarios', '...",1200000
3,fotocasa,20,Piso en venta,https://www.fotocasa.es/es/comprar/vivienda/ma...,https://static.inmofactory.com/statics/inmofac...,España,Vicálvaro,Valdebernardo - Valderribas,,28032,...,136,G,,Gas Natural,Muy bien,20 a 30 años,11,0,"{'1': 'Aire acondicionado', '2': 'Armarios', '...",380000
4,fotocasa,20,Piso en venta en Avenida Monasterio de Silos,https://www.fotocasa.es/es/comprar/vivienda/ma...,https://static.inmofactory.com/images/inmofact...,España,Fuencarral - El Pardo,Montecarmelo,Avenida Monasterio de Silos,28049,...,107,E,,,,10 a 20 años,9,0,"{'1': 'Aire acondicionado', '2': 'Armarios', '...",500000
5,fotocasa,20,Piso en venta en De la Virgen del Puerto,https://www.fotocasa.es/es/comprar/vivienda/ma...,https://static.inmofactory.com/images/inmofact...,España,Arganzuela,Imperial,De la Virgen del Puerto,28005,...,108,G,,Electricidad,Muy bien,50 a 70 años,8,0,"{'10': 'Terraza', '11': 'Trastero', '131': 'Co...",510000
6,fotocasa,20,Piso en venta en De Lope de Rueda,https://www.fotocasa.es/es/comprar/vivienda/ma...,https://static.inmofactory.com/images/inmofact...,España,Retiro,Ibiza de Madrid,De Lope de Rueda,28009,...,80,G,,Electricidad,A reformar,70 a 100 años,6,0,{'28': 'Serv. portería'},390000
7,fotocasa,20,Piso en venta en Francisco Silvela,https://www.fotocasa.es/es/comprar/vivienda/ma...,https://static.inmofactory.com/images/inmofact...,España,Barrio de Salamanca,Lista,Francisco Silvela,28028,...,82,E,,,,,9,0,"{'1': 'Aire acondicionado', '2': 'Armarios', '...",450000
8,fotocasa,20,Piso en venta en Calle Lérida,https://www.fotocasa.es/es/comprar/vivienda/ma...,https://static.inmofactory.com/images/inmofact...,España,Tetuán,Cuatro Caminos - Azca,Calle Lérida,28020,...,97,G,,Electricidad,Muy bien,50 a 70 años,7,0,"{'1': 'Aire acondicionado', '131': 'Cocina Equ...",365000
9,fotocasa,20,Piso en venta en Calle Bahía de Almería,https://www.fotocasa.es/es/comprar/vivienda/ma...,https://static.inmofactory.com/images/inmofact...,España,Barajas,Corralejos - Campo de las Naciones,Calle Bahía de Almería,28042,...,114,G,,Electricidad,Muy bien,20 a 30 años,3,0,"{'7': 'Jardín Privado', '17': 'Piscina', '28':...",429000


In [15]:
fc_scraping.data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 330 entries, 0 to 329
Data columns (total 31 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   origin             330 non-null    object
 1   page               330 non-null    int64 
 2   title              330 non-null    object
 3   link               330 non-null    object
 4   image_url          330 non-null    object
 5   country            330 non-null    object
 6   district           330 non-null    object
 7   neighborhood       330 non-null    object
 8   street             330 non-null    object
 9   zipCode            330 non-null    object
 10  province           330 non-null    object
 11  buildingType       330 non-null    object
 12  clientAlias        330 non-null    object
 13  latitude           330 non-null    object
 14  longitude          330 non-null    object
 15  isNewConstruction  330 non-null    object
 16  rooms              330 non-null    object
 1

## Modo solo de lectura de archivos

In [16]:
%%time
fc_scraping_fm = fotocasa_scraping(download = False, files_mode = True)

> Modo lectura activado.
> Done!
CPU times: user 580 ms, sys: 39.7 ms, total: 619 ms
Wall time: 847 ms


In [17]:
fc_scraping_fm.data.shape

(330, 31)

In [18]:
fc_scraping_fm.data.head(10)

,origin,page,title,link,image_url,country,district,neighborhood,street,zipCode,...,surface,energyCertificate,hotWater,heating,conservationState,antiquity,floor,surfaceLand,otherFeatures,price
0,fotocasa,23,Piso en venta,https://www.fotocasa.es/es/comprar/vivienda/ma...,https://static.inmofactory.com/images/inmofact...,España,Centro,Cortes - Huertas,,28012,...,38,G,Gas Natural,Gas Natural,Bien,+ 100 años,0,0,"{'1': 'Aire acondicionado', '2': 'Armarios', '...",250000
1,fotocasa,23,Piso en venta en Calle del Cerro del Campo,https://www.fotocasa.es/es/comprar/vivienda/ma...,https://static.inmofactory.com/images/inmofact...,España,Vicálvaro,El Cañaveral - Los Berrocales,Calle del Cerro del Campo,28052,...,73,B,,,Casi nuevo,,7,0,{},276000
2,fotocasa,23,Casa o chalet en venta,https://www.fotocasa.es/es/comprar/vivienda/ma...,https://static.inmofactory.com/images/inmofact...,España,Fuencarral - El Pardo,Mirasierra,,28034,...,340,G,,,Muy bien,10 a 20 años,3,0,"{'1': 'Aire acondicionado', '7': 'Jardín Priva...",1595000
3,fotocasa,23,Piso en venta en Ánsar,https://www.fotocasa.es/es/comprar/vivienda/ma...,https://static.inmofactory.com/images/inmofact...,España,Latina,Los Cármenes,Ánsar,28047,...,48,G,,,,50 a 70 años,0,0,"{'1': 'Aire acondicionado', '10': 'Terraza'}",115000
4,fotocasa,23,Piso en venta en Salsipuedes,https://www.fotocasa.es/es/comprar/vivienda/ma...,https://static.inmofactory.com/images/inmofact...,España,Villaverde,San Andrés,Salsipuedes,28021,...,63,E,,,,+ 100 años,0,0,{},156000
5,fotocasa,23,Piso en venta en Lagasca,https://www.fotocasa.es/es/comprar/vivienda/ma...,https://static.inmofactory.com/images/inmofact...,España,Barrio de Salamanca,Castellana,Lagasca,28006,...,241,G,,,,,6,0,"{'1': 'Aire acondicionado', '3': 'Calefacción'...",1800000
6,fotocasa,23,Piso en venta,https://www.fotocasa.es/es/comprar/vivienda/ma...,https://static.inmofactory.com/images/inmofact...,España,Arganzuela,Acacias,,28005,...,110,G,,,,,0,0,"{'1': 'Aire acondicionado', '2': 'Armarios', '...",530000
7,fotocasa,23,Planta baja en venta en Calle de Juan de Vera,https://www.fotocasa.es/es/comprar/vivienda/ma...,https://static.inmofactory.com/images/inmofact...,España,Arganzuela,Delicias,Calle de Juan de Vera,28045,...,42,G,,Electricidad,,70 a 100 años,0,0,{},150000
8,fotocasa,23,Piso en venta en Malcampo,https://www.fotocasa.es/es/comprar/vivienda/ma...,https://static.inmofactory.com/images/inmofact...,España,Chamartín,Ciudad Jardín,Malcampo,28002,...,84,E,,,,,6,0,{},325000
9,fotocasa,23,Piso en venta en Esperanza,https://www.fotocasa.es/es/comprar/vivienda/ma...,https://static.inmofactory.com/images/inmofact...,España,Arganzuela,Acacias,Esperanza,28005,...,47,D,,,,,0,0,"{'1': 'Aire acondicionado', '2': 'Armarios', '...",215000


In [19]:
fc_scraping_fm.data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 330 entries, 0 to 329
Data columns (total 31 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   origin             330 non-null    object
 1   page               330 non-null    object
 2   title              330 non-null    object
 3   link               330 non-null    object
 4   image_url          330 non-null    object
 5   country            330 non-null    object
 6   district           330 non-null    object
 7   neighborhood       330 non-null    object
 8   street             330 non-null    object
 9   zipCode            330 non-null    object
 10  province           330 non-null    object
 11  buildingType       330 non-null    object
 12  clientAlias        330 non-null    object
 13  latitude           330 non-null    object
 14  longitude          330 non-null    object
 15  isNewConstruction  330 non-null    object
 16  rooms              330 non-null    object
 1